In [1]:
from Seva import Seva
# import mysql.connector
obj = Seva()

In [2]:
list_cat = [['properti', 'tips-n-rekomendasi'], ['otomotif', 'keuangan'], ['otomotif', 'hobi-komunitas']
            , ['otomotif', 'editors-pick'], ['otomotif', 'modifikasi'], ['otomotif', 'review-otomotif']
            , ['otomotif', 'tips-rekomendasi'], ['otomotif', 'berita-terbaru'], ['otomotif', 'travel-lifestyle']
           , ['otomotif', 'berita-otomotif']]

In [3]:
for cat in list_cat:
    obj.getAllBerita([], 1, cat[1], cat[0])

page  1
https://www.seva.id/properti/blog/category/tips-n-rekomendasi/page/1
page  1
https://www.seva.id/otomotif/blog/category/keuangan/page/1
page  1
https://www.seva.id/otomotif/blog/category/hobi-komunitas/page/1
page  1
https://www.seva.id/otomotif/blog/category/editors-pick/page/1
page  1
https://www.seva.id/otomotif/blog/category/modifikasi/page/1
page  1
https://www.seva.id/otomotif/blog/category/review-otomotif/page/1
page  1
https://www.seva.id/otomotif/blog/category/tips-rekomendasi/page/1
page  1
https://www.seva.id/otomotif/blog/category/berita-terbaru/page/1
page  1
https://www.seva.id/otomotif/blog/category/travel-lifestyle/page/1
page  1
https://www.seva.id/otomotif/blog/category/berita-otomotif/page/1
